In [1]:
!pip install lightly -q

In [2]:
#dependencies for file management and data visualisation
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nibabel as nib
import re
from tqdm import tqdm
import pandas as pd
import gc
import psutil
from matplotlib.animation import FuncAnimation
import seaborn as sns
from IPython.display import HTML
import cv2

#pretraining dependencies
from lightly.data import LightlyDataset

#pretraining of SIMCLR MODEL
from lightly.transforms.simclr_transform import SimCLRTransform
from lightly.loss import NTXentLoss
from lightly.models.modules.heads import SimCLRProjectionHead

#pretraining of BarlowTwins MODEL
from lightly.loss import BarlowTwinsLoss
from lightly.models.modules import BarlowTwinsProjectionHead
from lightly.transforms.byol_transform import (
    BYOLTransform,
    BYOLView1Transform,
    BYOLView2Transform,
)

#pretraining on SWaV MODEL
from lightly.loss import SwaVLoss
from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes
from lightly.transforms.swav_transform import SwaVTransform


#fine-tuning the models
import torch
import torch.nn as nn
import torch.nn.functional as F


import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader , random_split
import pytorch_lightning as pl
import torchvision
from torchvision import transforms
from pytorch_lightning import seed_everything

#evaluating the models
from sklearn.metrics import log_loss

seed_value = 50

torch.manual_seed(seed_value)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)

seed_everything(42)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


42

GETTING ALL THE MRI IMAGES INTO ONE SINGLE FOLDER

In [3]:
final_loc='/kaggle/data'
c=0

In [4]:
data_path='/kaggle/input/ct-scans-2/CT_img_1'

os.makedirs(final_loc,exist_ok=True)
for file in os.listdir(data_path):
    c+=1
    
    file_path=os.path.join(data_path,file)
    ext=file[-4:]
    if(ext=='jpeg'):
        ext='.jpeg'
    new_filename=str(c)+ext
    final_path=os.path.join(final_loc,new_filename)
    
    shutil.copy(file_path, final_path)
    
    
    

In [5]:
counter=0
for i in os.listdir(final_loc):
    counter+=1
print(c,counter)

2987 2987


MAKING THE SIMCLR MODEL NOW

In [6]:
input_size=32
SC_Imagenet_transform = SimCLRTransform(input_size=input_size)

# Create a dataset from your image folder.
SC_Imagenet_dataset = LightlyDataset(
    input_dir=final_loc,
    transform=SC_Imagenet_transform,
)

# Build a PyTorch dataloader.
SC_Imagenet_dataloader = torch.utils.data.DataLoader(
    SC_Imagenet_dataset,                # Pass the dataset to the dataloader.
    batch_size=16,         # A large batch size helps with learning.
    shuffle=True,           # Shuffling is important!
    num_workers=4
)

In [7]:
max_epochs=25

class SimCLR(pl.LightningModule):
    def __init__(self):
        super().__init__()
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])

        
        self.projection_head = SimCLRProjectionHead(512,512,2048)
        #1st parameter -> input size (from the last layer of the resnet backbone)
        #2nd parameter -> hidden size
        #3rd parameter -> output size

        self.criterion = NTXentLoss()

    def forward(self, x):
        h = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(h)
        return z

    def training_step(self, batch, batch_idx):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(), lr=6e-2, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim],[scheduler]
    
torch.cuda.empty_cache()
SimCLR_ImagenetPT_model = SimCLR()
trainer = pl.Trainer(max_epochs=max_epochs, devices=1, accelerator="gpu")
trainer.fit(SimCLR_ImagenetPT_model, SC_Imagenet_dataloader)

Training: |          | 0/? [00:00<?, ?it/s]

MAKING THE BARLOWTWINS MODEL NOW

In [8]:
BT_Imagenet_transform = BYOLTransform(
    view_1_transform=BYOLView1Transform(input_size=32, gaussian_blur=0.0),
    view_2_transform=BYOLView2Transform(input_size=32, gaussian_blur=0.0),
)

BT_Imagenet_dataset = LightlyDataset(
    input_dir=final_loc,
    transform=BT_Imagenet_transform,
)

BT_Imagenet_dataloader = torch.utils.data.DataLoader(
    BT_Imagenet_dataset,   
    batch_size=16,         
    shuffle=True,           
    num_workers=4
)

In [9]:
max_epochs=25

class BarlowTwins(pl.LightningModule):
    def __init__(self):
        super().__init__()
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])
        self.projection_head = BarlowTwinsProjectionHead(512, 2048, 2048)
        self.criterion = BarlowTwinsLoss()

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_index):
        (x0, x1) = batch[0]
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(self.parameters(), lr=0.06)
        return optim
        


In [10]:
torch.cuda.empty_cache()
#resnet = torchvision.models.resnet18()
BT_ImagenetPT_model = BarlowTwins()
trainer = pl.Trainer(max_epochs=max_epochs, devices=1, accelerator="gpu")
trainer.fit(BT_ImagenetPT_model, BT_Imagenet_dataloader)

Training: |          | 0/? [00:00<?, ?it/s]

PRETRAINING OVER SWAV MODEL NOW

In [11]:
SW_Imagenet_transform = SwaVTransform()
# we ignore object detection annotations by setting target_transform to return 0
SW_Imagenet_dataset = LightlyDataset(
    input_dir=final_loc,
    transform=SW_Imagenet_transform,
)
# or create a dataset from a folder containing images or videos:
# dataset = LightlyDataset("path/to/folder")

SW_Imagenet_dataloader = torch.utils.data.DataLoader(
    SW_Imagenet_dataset,   
    batch_size=16,         
    shuffle=True,           
    num_workers=4
)


In [12]:
max_epochs=15
class SwaV(pl.LightningModule):
    def __init__(self):
        super().__init__()
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])
        self.projection_head = SwaVProjectionHead(512, 512, 128)
        self.prototypes = SwaVPrototypes(128, n_prototypes=512)
        self.criterion = SwaVLoss()

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        x = self.projection_head(x)
        x = nn.functional.normalize(x, dim=1, p=2)
        p = self.prototypes(x)
        return p

    def training_step(self, batch, batch_idx):
        self.prototypes.normalize()
        views = batch[0]
        multi_crop_features = [self.forward(view.to(self.device)) for view in views]
        high_resolution = multi_crop_features[:2]
        low_resolution = multi_crop_features[2:]
        loss = self.criterion(high_resolution, low_resolution)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(), lr=0.001)
        return optim

torch.cuda.empty_cache()
SW_ImagenetPT_model = SwaV()
trainer = pl.Trainer(max_epochs=max_epochs, devices=1, accelerator="gpu")
trainer.fit(SW_ImagenetPT_model, SW_Imagenet_dataloader)

Training: |          | 0/? [00:00<?, ?it/s]

NOW THAT THE PRETRAINING IS DONE, WE DEFINE THE TRAIN AND TEST DATALOADER FOR THE FINE-TUNING TASK

In [13]:
def one_hot(n):
    index={0:0,0.5:1,1:2,2:3}
    #print(index)
    #print(index[n])
    y_new=np.zeros(4)
    y_new[index[n]]=1
    #print(y_new)
    return y_new

In [14]:
c=0
df_new = pd.DataFrame(columns=['IMG', 'CDR'])
def make_dataset(path,cdr):
    
    global c, df_new
    
    for i in os.listdir(path):
        c+=1
        image = cv2.imread(os.path.join(path,i))
        image_np = np.array(image)
        image_tensor = transforms.ToTensor()(image_np)
        padded_image = transforms.CenterCrop((224, 224))(transforms.Pad(28)(image_tensor))
        #print(padded_image.shape)
        '''plt.imshow(padded_image.permute(1, 2, 0))
        print('padded_image',padded_image.shape)'''  
        
        new_entry = {
            'IMG': padded_image,
            'CDR': cdr
        } 
        df_new = pd.concat([df_new, pd.DataFrame([new_entry])], ignore_index=True)
        #print(i)


make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented',1)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/ModerateDemented',2)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/NonDemented',0)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/VeryMildDemented',0.5)
        
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/MildDemented',1)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/ModerateDemented',2)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/NonDemented',0)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/VeryMildDemented',0.5)


In [15]:
print(c,df_new.shape)
print(df_new['IMG'][0].shape)

6400 (6400, 2)
torch.Size([3, 224, 224])


In [16]:
x=np.stack(df_new['IMG'].tolist())
y=df_new['CDR']

y_one_hot=[]
for i in y:
    y_one_hot.append(one_hot(i)) 

In [17]:
print(type(x))

<class 'numpy.ndarray'>


In [18]:
x=torch.tensor(x)
y=torch.Tensor(y_one_hot)

dataset=TensorDataset(x,y)

train_size=int(0.6*len(x))
val_size=int(0.2*len(x))
temp_size=2*val_size

#print(train_size,val_size,temp_size)

train_dataset, temp_dataset = random_split(dataset, [train_size, temp_size])
val_dataset,test_dataset=random_split(temp_dataset, [val_size,val_size])

# Define batch size
#batch_size = 8

# Create a DataLoader for shuffling and batching
'''train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=1,shuffle=True)'''

/tmp/ipykernel_47/2762140771.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  y=torch.Tensor(y_one_hot)


'train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)\nval_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)\ntest_dataloader=DataLoader(test_dataset,batch_size=1,shuffle=True)'

In [19]:
def train_and_val(backbone):
    
    #training the models first on train_dataloader
    
    models=[]
    for i in range(2):
        models.append(ResnetSingleChannel(backbone,4))
        
   
    
    batch_sizes=[64,32]
    
    for model_no in range(2):
        print(f'TRAINING MODEL {model_no+1}')
        print('--------------------------------------------------------------')
        fine_tune_opt = optim.Adam(models[model_no].parameters(), lr=0.001, weight_decay=0.0001)
        ft_loss_fn= nn.CrossEntropyLoss()
        
        batch_size=batch_sizes[model_no]
        
        train_dataloader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)
        

        num_epochs = 8
        total_steps=train_size//batch_size

        for epoch in range(num_epochs):
            i=0

            for x_batch,y_batch in train_dataloader:
                i+=1
                outputs=models[model_no](torch.Tensor(x_batch))
                
                '''print('out',outputs.shape)
                print('y',y_batch.shape)
                print('out',outputs)
                print('y',y_batch)'''
                
                loss = ft_loss_fn(outputs, y_batch)
                fine_tune_opt.zero_grad()
                loss.backward()
                fine_tune_opt.step()

                if (i+1) % 20 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')
    
    # performing validation on validation data_loader
    best_model=-1
    best_loss=np.inf
    for model_no in range(2):
        
        loss_sum=0
        
        for x_batch,y_batch in val_dataloader:
            outputs=models[model_no](torch.Tensor(x_batch))
            loss = ft_loss_fn(outputs, y_batch)
            loss_sum+=loss.item()
            
        if(loss_sum<best_loss):
            best_model=model_no
            best_loss=loss_sum
    
    return models[best_model]
        
        

In [20]:
def softmax(x):
    return F.softmax(x, dim=-1)

def metrics(model):
    
    test_dataloader=DataLoader(test_dataset,batch_size=1,shuffle=True)
    
    y_pred_M=[]
    y_true_M=[]
    c=0
    logloss_sum=0
    
    for x_batch,y_batch in test_dataloader:
        
        c+=1
        y_pred=model(x_batch)

        '''y_pred_M.append(np.argmax(y_pred.detach().numpy()))
        y_true_M.append(np.argmax(y_batch.detach().numpy()))'''

        softmax_probs = softmax(y_pred)

        '''print(y_pred)
        print(softmax_probs)
        print(y_batch)'''

        logloss = log_loss(y_batch.detach().numpy(), softmax_probs.detach().numpy())

        logloss_sum+=logloss
        #print(logloss) 

    return(logloss_sum/c)

In [21]:
#MAKING A NEW NETWORK WHICH USES THE BACKBONE OF THE PRETRAINED MODEL, AND HAS AN INPUT LAYER TO TAKE IN A SINGLE CHANNEL IMAGE, AND THEN PASS INTO 
#BACKBONE, AND FINALLY OUTPUT LAYER WHICH PERFORMS REGRESSION

class ResnetSingleChannel(nn.Module):
    def __init__(self,backbone,channels):
        super(ResnetSingleChannel,self).__init__()
        self.input=nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1, bias=False)
        self.backbone=backbone
        self.output=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=512, out_features=256),
            nn.Linear(in_features=256, out_features=channels)
        )
        
    def forward(self,x):
        x=self.input(x)
        x=self.backbone(x)
        x=self.output(x)
        
        return x

**KEEPING THE BACKBONE UNFROZEN**

In [22]:
for layer in SimCLR_ImagenetPT_model.backbone:
    for param in layer.parameters():
        param.requires_grad = True
        
SC_MODEL=train_and_val(SimCLR_ImagenetPT_model.backbone)
SC_M_2=metrics(SC_MODEL)

TRAINING MODEL 1
--------------------------------------------------------------
Epoch [1/8], Step [20/60], Loss: 0.9522
Epoch [1/8], Step [40/60], Loss: 0.9032
Epoch [1/8], Step [60/60], Loss: 0.8346
Epoch [2/8], Step [20/60], Loss: 0.7374
Epoch [2/8], Step [40/60], Loss: 0.6542
Epoch [2/8], Step [60/60], Loss: 0.7728
Epoch [3/8], Step [20/60], Loss: 0.7502
Epoch [3/8], Step [40/60], Loss: 0.7464
Epoch [3/8], Step [60/60], Loss: 0.9391
Epoch [4/8], Step [20/60], Loss: 0.4308
Epoch [4/8], Step [40/60], Loss: 0.5457
Epoch [4/8], Step [60/60], Loss: 0.3963
Epoch [5/8], Step [20/60], Loss: 0.4343
Epoch [5/8], Step [40/60], Loss: 0.2523
Epoch [5/8], Step [60/60], Loss: 0.2744
Epoch [6/8], Step [20/60], Loss: 0.2748
Epoch [6/8], Step [40/60], Loss: 0.3079
Epoch [6/8], Step [60/60], Loss: 0.2419
Epoch [7/8], Step [20/60], Loss: 0.1854
Epoch [7/8], Step [40/60], Loss: 0.2664
Epoch [7/8], Step [60/60], Loss: 0.3010
Epoch [8/8], Step [20/60], Loss: 0.1360
Epoch [8/8], Step [40/60], Loss: 0.0294


In [23]:
for layer in BT_ImagenetPT_model.backbone:
    for param in layer.parameters():
        param.requires_grad = True
        
BT_MODEL=train_and_val(BT_ImagenetPT_model.backbone)
BT_M_2=metrics(BT_MODEL)

TRAINING MODEL 1
--------------------------------------------------------------
Epoch [1/8], Step [20/60], Loss: 1.1895
Epoch [1/8], Step [40/60], Loss: 1.1621
Epoch [1/8], Step [60/60], Loss: 1.1004
Epoch [2/8], Step [20/60], Loss: 1.0181
Epoch [2/8], Step [40/60], Loss: 1.4636
Epoch [2/8], Step [60/60], Loss: 1.2567
Epoch [3/8], Step [20/60], Loss: 1.1022
Epoch [3/8], Step [40/60], Loss: 1.0371
Epoch [3/8], Step [60/60], Loss: 1.0991
Epoch [4/8], Step [20/60], Loss: 1.0467
Epoch [4/8], Step [40/60], Loss: 1.1119
Epoch [4/8], Step [60/60], Loss: 1.1012
Epoch [5/8], Step [20/60], Loss: 1.1756
Epoch [5/8], Step [40/60], Loss: 1.1035
Epoch [5/8], Step [60/60], Loss: 0.9628
Epoch [6/8], Step [20/60], Loss: 0.9748
Epoch [6/8], Step [40/60], Loss: 0.9543
Epoch [6/8], Step [60/60], Loss: 0.9647
Epoch [7/8], Step [20/60], Loss: 0.8936
Epoch [7/8], Step [40/60], Loss: 1.0971
Epoch [7/8], Step [60/60], Loss: 1.1290
Epoch [8/8], Step [20/60], Loss: 1.0406
Epoch [8/8], Step [40/60], Loss: 1.0065


In [24]:
for layer in SW_ImagenetPT_model.backbone:
    for param in layer.parameters():
        param.requires_grad = True
        
SW_MODEL=train_and_val(SW_ImagenetPT_model.backbone)
SW_M_2=metrics(SW_MODEL)

TRAINING MODEL 1
--------------------------------------------------------------
Epoch [1/8], Step [20/60], Loss: 1.0218
Epoch [1/8], Step [40/60], Loss: 0.7916
Epoch [1/8], Step [60/60], Loss: 0.9214
Epoch [2/8], Step [20/60], Loss: 0.8782
Epoch [2/8], Step [40/60], Loss: 1.0232
Epoch [2/8], Step [60/60], Loss: 0.9126
Epoch [3/8], Step [20/60], Loss: 0.8705
Epoch [3/8], Step [40/60], Loss: 0.8530
Epoch [3/8], Step [60/60], Loss: 0.7327
Epoch [4/8], Step [20/60], Loss: 0.6741
Epoch [4/8], Step [40/60], Loss: 0.8561
Epoch [4/8], Step [60/60], Loss: 0.8215
Epoch [5/8], Step [20/60], Loss: 0.9003
Epoch [5/8], Step [40/60], Loss: 0.8323
Epoch [5/8], Step [60/60], Loss: 0.8250
Epoch [6/8], Step [20/60], Loss: 0.7093
Epoch [6/8], Step [40/60], Loss: 0.5818
Epoch [6/8], Step [60/60], Loss: 0.4924
Epoch [7/8], Step [20/60], Loss: 0.8866
Epoch [7/8], Step [40/60], Loss: 0.5955
Epoch [7/8], Step [60/60], Loss: 0.4672
Epoch [8/8], Step [20/60], Loss: 0.4573
Epoch [8/8], Step [40/60], Loss: 0.3895


In [25]:
print('SimCLR',SC_M_2)
print('Barlow',BT_M_2)
print('SwAV',SW_M_2)


SimCLR 1.2063910560350524
Barlow 1.0927559791338648
SwAV 1.2759309196134094
